## Initialize the ckan environment and requests session

In [ ]:
from os import path, environ
import requests
from dataflows import Flow, load
from datapackage_pipelines_ckanext.helpers import get_plugin_configuration

config = get_plugin_configuration('odata_org_il')
data_path = config['data_path']

CKAN_API_KEY = environ.get('CKAN_API_KEY')
CKAN_URL = environ.get('CKAN_URL')
assert CKAN_API_KEY and CKAN_URL
CKAN_AUTH_HEADERS = {'Authorization': CKAN_API_KEY}
session = requests.session()
session.headers.update(CKAN_AUTH_HEADERS)

## Source entity data from foi site

In [ ]:
from dataflows import Flow, load
import yaml

def process(rows):
    for row in rows:
        if int(row['nid']) == 446:
            print(yaml.dump(row, default_flow_style=False, allow_unicode=True))
            yield row

source_entity = Flow(
    load(data_path+'/new_foi_offices/datapackage.json'),
    process
).results()[0][0]

## Matching ckan group from foi_groups_matching excel resource

In [ ]:
from dataflows import Flow, load
import yaml

def process(rows):
    for i, row in enumerate(rows):
        if row['entity_id'] == f'foi-office-{source_entity["nid"]}':
            yield row

foi_group_matching_resource = Flow(load(data_path+'/foi_groups_matching/datapackage.json')).results()[0][0]
foi_group_matching_source_entity = [row for row in foi_group_matching_resource if row['entity_id'] == f'foi-office-{source_entity["nid"]}'][0]
print(yaml.dump(foi_group_matching_source_entity, default_flow_style=False, allow_unicode=True))

## Load existing entities and find matching group

In [ ]:
from dataflows import Flow, load
from load_existing_entities import get_existing_entities_resource, get_existing_entities_resource_descriptor
from collections import defaultdict

stats = defaultdict(int)
existing_entities_resource = Flow(load(({'resources': [get_existing_entities_resource_descriptor()]}, 
                                        [get_existing_entities_resource(stats)]))
                                 ).results()[0][0]
existing_entity = [row for row in existing_entities_resource if row['group_id'] == foi_group_matching_source_entity['Column3']][0]
print(yaml.dump(existing_entity, default_flow_style=False, allow_unicode=True))
print(f'num existing entities = {len(existing_entities)}')
print(dict(stats))

## Run dry run to update_foi_offices_entities manually only for this group

In [ ]:
from update_foi_offices_entities import get_foi_offices_resource, get_existing_entities, get_foi_groups_matching
from collections import defaultdict
import yaml

stats = defaultdict(int)

existing_entities = {}
for row in get_existing_entities(existing_entities_resource, existing_entities, stats):
    pass

for row in get_foi_groups_matching(foi_group_matching_resource, existing_entities, stats):
    pass

for row in get_foi_offices_resource([source_entity], existing_entities, stats, True):
    print(yaml.dump(row, default_flow_style=False, allow_unicode=True))

## Before updating - save the group datasets, otherwise they will be disconnected from group

In [ ]:
raise NotImplementedError()

## Do the update

In [ ]:
for row in get_foi_offices_resource([source_entity], existing_entities, stats, False):
    print(yaml.dump(row, default_flow_style=False, allow_unicode=True))

## Restore the datasets

In [ ]:
from os import path, environ
import requests
from dataflows import Flow, load
from datapackage_pipelines_ckanext.helpers import get_plugin_configuration

def restore_group_datasets(row):
    group_id = row['group_id']
    if group_id == existing_entity['group_id']:
        for dataset_id in row['dataset_ids']:
            res = session.post('{}/api/3/action/member_create'.format(CKAN_URL),
                               json=dict(id=group_id,
                                         object=dataset_id,
                                         object_type='package',
                                         capacity='')).json()
            assert res and res['success']

Flow(
    load(path.join(data_path, 'dump_group_datasets/datapackage.json'), resources=['group_datasets']),
    restore_group_datasets
).process()
